<a href="https://colab.research.google.com/github/GbrlOl/whisper-fast/blob/main/Whisper_Large_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalamos las dependencias

In [1]:
!pip install --upgrade -qqq git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Cargamos el Whisper Large-V3

In [6]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


# Especificamos la ruta en donde se ubica el audio para transcribir

In [3]:
result = pipe("/content/CLASE_9.mp3", generate_kwargs={"language": "spanish"})
result

{'text': ' bien, entonces ahora partió esto buenas tardes no sé si está grabando sí, buenas tardes estudiantes espero que estén súper bien vamos hoy día, como les comenté en la clase pasada, hoy día la idea de la clase de hoy es ver algún problema tipo prueba, para que ustedes sepan y entiendan qué es lo que estamos trabajando me imagino que ya han estudiado recuerden que en el aula virtual ya se les dejó el problema 1 de la prueba, así que no voy a tocar nada de eso hoy día, vamos a ver otras cosas que no sean de lo que les pregunto en esa pregunta, y eso primero, principalmente eso. Así que recordar que ya vimos la materia, ya tenemos la materia completa, recordar que ya vimos la materia ya tenemos la materia completa pasamos la toda la unidad, la clase pasada terminamos de ver la parte de TN y Norton y vimos también TLEGEN, potencia, hicimos problemas y no sé si ustedes tienen dudas o consultas hay 18 alumnos ahora espero que estén súper bien bien, entonces vamos a partir, abrí si u

# Convertir a un archivo de texto

In [21]:
def convertir_a_texto(transcripcion: str) -> str:
  with open("transcripción_audio.txt", 'w', encoding='utf-8') as archivo:
    archivo.write(transcripcion['text'].strip())

  return print("La transcripción quedó lista")

In [22]:
convertir_a_texto(result)

La transcripción quedó lista


# En Construcción

In [17]:
result1['text']

" Hola a todos, Jerry aquí, y en este video estamos emocionados de presentar a Wenqi, que hablará de 12 puntos de dolor de RAG y soluciones propuestas en Lama Index. Como mucha gente sabe, que está construyendo RAG hoy en día, construir RAG de producción de performance puede ser bastante desafiante, y puede ser muy útil pinponar los puntos de error específicos a través de la pipeline de RAG, así como identificar las soluciones para cada uno de estos puntos de falla específicos a través de la red de rango, así como identificar las soluciones para cada de estos puntos de dolor. Y así que, una vez que estará pasando por un artículo muy popular hacia la ciencia de datos que escribió hace unos días atrás y también simplemente ir a través de él visualmente y mostrar algunos ejemplos. Y así que, sin más detalles, ¡pongámoslo! Gracias Jerry, sí, feliz de estar aquí. Así que hoy vamos a entrar en los 12 puntos de pinta de la RAC y las soluciones propuestas. Realmente está resolviendo los desafí

In [10]:
result.keys()

dict_keys(['text', 'chunks'])


In [13]:
with open("transcripcion.txt", "w", encoding='utf-8') as archivo:
  archivo.write(f"{result['text']}")

In [15]:
result['text'].strip()

"Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!"

In [17]:
result['chunks']

[{'timestamp': (0.0, 2.2), 'text': ' bien, entonces'},
 {'timestamp': (2.2, 4.76), 'text': ' ahora partió esto'},
 {'timestamp': (4.76, 6.04), 'text': ' buenas tardes'},
 {'timestamp': (6.04, 7.1), 'text': ' no sé si está grabando'},
 {'timestamp': (7.1, 11.22), 'text': ' sí, buenas tardes estudiantes'},
 {'timestamp': (11.22, 12.22), 'text': ' espero que estén súper bien'},
 {'timestamp': (12.22, 15.1),
  'text': ' vamos hoy día, como les comenté en la clase'},
 {'timestamp': (15.1, 16.8), 'text': ' pasada, hoy día la idea'},
 {'timestamp': (16.8, 19.22),
  'text': ' de la clase de hoy es ver algún problema'},
 {'timestamp': (19.22, 20.94), 'text': ' tipo prueba, para que ustedes sepan'},
 {'timestamp': (20.94, 22.68),
  'text': ' y entiendan qué es lo que estamos trabajando'},
 {'timestamp': (22.68, 28.66),
  'text': ' me imagino que ya han estudiado recuerden que en el aula virtual ya se les dejó el problema 1 de la prueba,'},
 {'timestamp': (29.24, 35.64),
  'text': ' así que no vo

In [19]:
def convert_to_vtt(data_list, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('WEBVTT\n\n')

        for entry in data_list:
            timestamp = entry.get('timestamp', '')
            text = entry.get('text', '')

            if timestamp and text:
                formatted_time = f"{timestamp[:2]}:{timestamp[2:4]}:{timestamp[4:6]}.{timestamp[6:]}"
                f.write(f"{formatted_time}\n{text}\n\n")

# Ejemplo de lista de diccionarios de entrada
data_list = [
    {'timestamp': '010203000', 'text': 'Hola, mundo!'},
    {'timestamp': '020405000', 'text': 'Esto es un ejemplo.'},
    # Añade más entradas según sea necesario
]

# Llama a la función para convertir la lista a VTT
convert_to_vtt(result['chunks'], 'output.vtt')


In [ ]:
result

{'text': ' hola amigos como están acá perfecto nuevamente en youtube y vamos a aplicar la misma técnica que usé para las redes sociales para instalar para poder crecer ahí simplemente dar vídeos sencillos sin mucho color sin mucha edición ahí que vayan fluyendo de forma natural y hacerlo de forma muy constante y periódica como lo que usted tiene que hacer si es que quiere lograr algo bueno en su vida mejor hecho que perfecto me enseñó un buen amigo un buen dicho es y amigos yo hablando de cosas que son útiles de cosas que son importantes en la una de ellas es alimentarse de forma adecuada y acá también el dicho este de mejor hecho que perfecto aplica bien usted busque los alimentos bien naturales de origen natural de densidad nutricional alta prefiera los de esa manera se va a mantener siempre en forma con un buen porcentaje de grasa, con buena salud, con un buen balance de minerales de vitaminas y van a estar generalmente si usted ha hecho una miradita más o menos al plato, con videos

In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [23]:
!pip install -qqq pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00


In [2]:
from pytube import YouTube

In [3]:
url = input('Pegar link y enter: ')

Pegar link y enter: https://www.youtube.com/watch?v=EBpT_cscTis


In [4]:
yt_url = YouTube(url)
audio_stream = yt_url.streams.filter(only_audio=True).first()
output_path = '/content'
filename = 'audio.mp3'
audio_stream.download(output_path=output_path, filename=filename)
print(f'El audio quedó en el colab en {output_path}/{filename}')

El audio quedó en el colab en /content/audio.mp3


In [7]:
result1 = pipe("/content/audio.mp3", generate_kwargs={"language": "spanish"})
result1

{'text': " Hola a todos, Jerry aquí, y en este video estamos emocionados de presentar a Wenqi, que hablará de 12 puntos de dolor de RAG y soluciones propuestas en Lama Index. Como mucha gente sabe, que está construyendo RAG hoy en día, construir RAG de producción de performance puede ser bastante desafiante, y puede ser muy útil pinponar los puntos de error específicos a través de la pipeline de RAG, así como identificar las soluciones para cada uno de estos puntos de falla específicos a través de la red de rango, así como identificar las soluciones para cada de estos puntos de dolor. Y así que, una vez que estará pasando por un artículo muy popular hacia la ciencia de datos que escribió hace unos días atrás y también simplemente ir a través de él visualmente y mostrar algunos ejemplos. Y así que, sin más detalles, ¡pongámoslo! Gracias Jerry, sí, feliz de estar aquí. Así que hoy vamos a entrar en los 12 puntos de pinta de la RAC y las soluciones propuestas. Realmente está resolviendo l

In [11]:
def convert_to_vtt(data_list, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('WEBVTT\n\n')

        for entry in data_list:
            if isinstance(entry, dict):
                start_time = entry.get('timestamp')[0]
                end_time = entry.get('timestamp')[1]
                text = entry.get('text', '')

                if start_time is not None and end_time is not None and text:
                    formatted_start_time = format_time(start_time)
                    formatted_end_time = format_time(end_time)
                    f.write(f"{formatted_start_time} --> {formatted_end_time}\n{text}\n\n")
            else:
                print(f"Entry {entry} is not a dictionary. Skipping...")

def format_time(time):
    hours = int(time // 3600)
    minutes = int((time % 3600) // 60)
    seconds = int(time % 60)
    milliseconds = int((time % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"


# Llama a la función para convertir la lista a VTT
convert_to_vtt(result1, 'llamaindex.vtt')

Entry text is not a dictionary. Skipping...
Entry chunks is not a dictionary. Skipping...


In [13]:
def convert_to_vtt(data):
    vtt_content = "WEBVTT\n\n"

    for item in data:
        start_time = "{:02d}:{:02d}:{:02d}.{:03d}".format(int(item['timestamp'][0] // 60),
                                                           int(item['timestamp'][0] % 60),
                                                           int((item['timestamp'][0] % 1) * 1000),
                                                           int(((item['timestamp'][0] % 1) * 1000) % 1000))
        end_time = "{:02d}:{:02d}:{:02d}.{:03d}".format(int(item['timestamp'][1] // 60),
                                                         int(item['timestamp'][1] % 60),
                                                         int((item['timestamp'][1] % 1) * 1000),
                                                         int(((item['timestamp'][1] % 1) * 1000) % 1000))

        vtt_content += f"{start_time} --> {end_time}\n{item['text']}\n\n"

    return vtt_content

# data = [
#     {'timestamp': (0.0, 4.7), 'text': 'Hola a todos, Jerry aquí, y en este video estamos emocionados de presentar a Wenqi,'},
#     {'timestamp': (4.84, 9.94), 'text': 'que hablará de 12 puntos de dolor de RAG y soluciones propuestas en Lama Index.'},
#     {'timestamp': (10.48, 14.04), 'text': 'Como mucha gente sabe, que está construyendo RAG hoy en día,'},
#     {'timestamp': (14.68, 17.72), 'text': 'construir RAG de producción de performance puede ser bastante desafiante,'},
#     {'timestamp': (18.24, 22.6), 'text': 'ejemplo de último elemento sin texto completo.'}
# ]

vtt_content = convert_to_vtt(result1['chunks'])

with open("llamaindex.vtt", "w") as file:
    file.write(vtt_content)

print("Archivo .vtt creado exitosamente.")


Archivo .vtt creado exitosamente.
